O primeiro passo para nosso projeto é decidir quais técnicas iremos usar para o nosso metodo classico de classificação. Optamos por usar 

Começamos por ler nosso csv de treino e teste e separar o texto e as emoções, criaremos também a coluna para neutro, para quando nenhuma das emoções for identificada. 

In [29]:
import pandas as pd

data = pd.read_csv("treino/ptbr.csv")
teste = pd.read_csv("teste/ptbr.csv")


data["neutral"] = (data[["anger", "disgust", "fear", "joy", "sadness", "surprise"]].sum(axis=1) == 0).astype(int)
teste["neutral"] = (teste[["anger", "disgust", "fear", "joy", "sadness", "surprise"]].sum(axis=1) == 0).astype(int)

texto = data["text"]
emocao = data[["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]]

texto_teste = teste["text"]
emocao_teste = teste[["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]]



Iremos transformar o texto em vetores usando o sklearn, e depois o Sklearn fará a Regressão Lógica, após isso faremos o fit para o treino e por fim a predição do conjunto de teste

In [30]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

features = FeatureUnion([
    ('count', CountVectorizer()),
    ('tfidf', TfidfVectorizer())
])

ordem = Pipeline([
    ('features', features),
    ('clf', OneVsRestClassifier(LogisticRegression(class_weight='balanced')))
])

ordem_sem_bal = Pipeline([
    ('features', features),
    ('clf', OneVsRestClassifier(LogisticRegression()))
])

ordem.fit(texto, emocao)
ordem_sem_bal.fit(texto,emocao)

probas = ordem.predict_proba(texto_teste)

threshold = 0.3141592653589793
predi = (probas >= threshold).astype(int)
predi_sem_bal = ordem_sem_bal.predict(texto_teste)


Para comparação estamos trabalhando com 2 modelos, um onde apenas aplicamos o que foi aprendido com o conjunto de treino na nossa base de testes, e outro onde aplicamos técnicas para possivelmente melhorar nossos resultados, essas técnicas são, o blaceamento de classes minoritárias, dando mais peso a elas, e um threshold diferente do padrão, aqui foi escolhido o valor de pi como um valor aleátorio.

In [31]:
from sklearn.metrics import classification_report

print(classification_report(emocao_teste, predi, target_names=emocao_teste.columns))
print(classification_report(emocao_teste, predi_sem_bal, target_names=emocao_teste.columns))



              precision    recall  f1-score   support

       anger       0.48      0.75      0.58       699
     disgust       0.18      0.24      0.21        71
        fear       0.20      0.24      0.22       139
         joy       0.47      0.72      0.57       555
     sadness       0.35      0.59      0.44       343
    surprise       0.13      0.24      0.17       157
     neutral       0.41      0.74      0.52       616

   micro avg       0.40      0.65      0.49      2580
   macro avg       0.32      0.50      0.39      2580
weighted avg       0.40      0.65      0.49      2580
 samples avg       0.44      0.67      0.50      2580

              precision    recall  f1-score   support

       anger       0.64      0.43      0.51       699
     disgust       0.00      0.00      0.00        71
        fear       0.60      0.04      0.08       139
         joy       0.76      0.41      0.53       555
     sadness       0.60      0.25      0.35       343
    surprise       0.20 

C:\Users\theod\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\theod\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
